# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f6f54bbc4f0>
├── 'a' --> tensor([[-0.2540,  2.5304,  0.8798],
│                   [ 0.1927, -0.6284, -1.0389]])
└── 'x' --> <FastTreeValue 0x7f6f54bbce50>
    └── 'c' --> tensor([[-0.0393,  1.6880, -1.1207, -0.1781],
                        [ 1.3211,  0.3347, -0.8080, -0.8914],
                        [ 0.5762, -0.4021,  1.0313, -0.9257]])

In [4]:
t.a

tensor([[-0.2540,  2.5304,  0.8798],
        [ 0.1927, -0.6284, -1.0389]])

In [5]:
%timeit t.a

64.5 ns ± 0.048 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f6f54bbc4f0>
├── 'a' --> tensor([[-0.2503,  0.3040, -1.2587],
│                   [-1.0292,  1.2959,  0.2614]])
└── 'x' --> <FastTreeValue 0x7f6f54bbce50>
    └── 'c' --> tensor([[-0.0393,  1.6880, -1.1207, -0.1781],
                        [ 1.3211,  0.3347, -0.8080, -0.8914],
                        [ 0.5762, -0.4021,  1.0313, -0.9257]])

In [7]:
%timeit t.a = new_value

64.8 ns ± 0.038 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2540,  2.5304,  0.8798],
               [ 0.1927, -0.6284, -1.0389]]),
    x: Batch(
           c: tensor([[-0.0393,  1.6880, -1.1207, -0.1781],
                      [ 1.3211,  0.3347, -0.8080, -0.8914],
                      [ 0.5762, -0.4021,  1.0313, -0.9257]]),
       ),
)

In [10]:
b.a

tensor([[-0.2540,  2.5304,  0.8798],
        [ 0.1927, -0.6284, -1.0389]])

In [11]:
%timeit b.a

54.9 ns ± 0.0375 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6183,  0.0348, -1.8972],
               [-0.4237,  0.6087, -0.6263]]),
    x: Batch(
           c: tensor([[-0.0393,  1.6880, -1.1207, -0.1781],
                      [ 1.3211,  0.3347, -0.8080, -0.8914],
                      [ 0.5762, -0.4021,  1.0313, -0.9257]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.241 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.14 µs ± 0.663 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.8 µs ± 22.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 429 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 587 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f6e8ef3b9d0>
├── 'a' --> tensor([[[-0.2540,  2.5304,  0.8798],
│                    [ 0.1927, -0.6284, -1.0389]],
│           
│                   [[-0.2540,  2.5304,  0.8798],
│                    [ 0.1927, -0.6284, -1.0389]],
│           
│                   [[-0.2540,  2.5304,  0.8798],
│                    [ 0.1927, -0.6284, -1.0389]],
│           
│                   [[-0.2540,  2.5304,  0.8798],
│                    [ 0.1927, -0.6284, -1.0389]],
│           
│                   [[-0.2540,  2.5304,  0.8798],
│                    [ 0.1927, -0.6284, -1.0389]],
│           
│                   [[-0.2540,  2.5304,  0.8798],
│                    [ 0.1927, -0.6284, -1.0389]],
│           
│                   [[-0.2540,  2.5304,  0.8798],
│                    [ 0.1927, -0.6284, -1.0389]],
│           
│                   [[-0.2540,  2.5304,  0.8798],
│                    [ 0.1927, -0.6284, -1.0389]]])
└── 'x' --> <FastTreeValue 0x7f6e8ef3b8e0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 138 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f6f56fb1a90>
├── 'a' --> tensor([[-0.2540,  2.5304,  0.8798],
│                   [ 0.1927, -0.6284, -1.0389],
│                   [-0.2540,  2.5304,  0.8798],
│                   [ 0.1927, -0.6284, -1.0389],
│                   [-0.2540,  2.5304,  0.8798],
│                   [ 0.1927, -0.6284, -1.0389],
│                   [-0.2540,  2.5304,  0.8798],
│                   [ 0.1927, -0.6284, -1.0389],
│                   [-0.2540,  2.5304,  0.8798],
│                   [ 0.1927, -0.6284, -1.0389],
│                   [-0.2540,  2.5304,  0.8798],
│                   [ 0.1927, -0.6284, -1.0389],
│                   [-0.2540,  2.5304,  0.8798],
│                   [ 0.1927, -0.6284, -1.0389],
│                   [-0.2540,  2.5304,  0.8798],
│                   [ 0.1927, -0.6284, -1.0389]])
└── 'x' --> <FastTreeValue 0x7f6f56bd71f0>
    └── 'c' --> tensor([[-0.0393,  1.6880, -1.1207, -0.1781],
                        [ 1.3211,  0.3347, -0.8080, -0.8914],
                 

In [23]:
%timeit t_cat(trees)

30.9 µs ± 66 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.1 µs ± 78 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2540,  2.5304,  0.8798],
                [ 0.1927, -0.6284, -1.0389]],
       
               [[-0.2540,  2.5304,  0.8798],
                [ 0.1927, -0.6284, -1.0389]],
       
               [[-0.2540,  2.5304,  0.8798],
                [ 0.1927, -0.6284, -1.0389]],
       
               [[-0.2540,  2.5304,  0.8798],
                [ 0.1927, -0.6284, -1.0389]],
       
               [[-0.2540,  2.5304,  0.8798],
                [ 0.1927, -0.6284, -1.0389]],
       
               [[-0.2540,  2.5304,  0.8798],
                [ 0.1927, -0.6284, -1.0389]],
       
               [[-0.2540,  2.5304,  0.8798],
                [ 0.1927, -0.6284, -1.0389]],
       
               [[-0.2540,  2.5304,  0.8798],
                [ 0.1927, -0.6284, -1.0389]]]),
    x: Batch(
           c: tensor([[[-0.0393,  1.6880, -1.1207, -0.1781],
                       [ 1.3211,  0.3347, -0.8080, -0.8914],
                       [ 0.5762, -0.4021,  1.0313, -0.9257]],
         

In [26]:
%timeit Batch.stack(batches)

81.1 µs ± 302 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2540,  2.5304,  0.8798],
               [ 0.1927, -0.6284, -1.0389],
               [-0.2540,  2.5304,  0.8798],
               [ 0.1927, -0.6284, -1.0389],
               [-0.2540,  2.5304,  0.8798],
               [ 0.1927, -0.6284, -1.0389],
               [-0.2540,  2.5304,  0.8798],
               [ 0.1927, -0.6284, -1.0389],
               [-0.2540,  2.5304,  0.8798],
               [ 0.1927, -0.6284, -1.0389],
               [-0.2540,  2.5304,  0.8798],
               [ 0.1927, -0.6284, -1.0389],
               [-0.2540,  2.5304,  0.8798],
               [ 0.1927, -0.6284, -1.0389],
               [-0.2540,  2.5304,  0.8798],
               [ 0.1927, -0.6284, -1.0389]]),
    x: Batch(
           c: tensor([[-0.0393,  1.6880, -1.1207, -0.1781],
                      [ 1.3211,  0.3347, -0.8080, -0.8914],
                      [ 0.5762, -0.4021,  1.0313, -0.9257],
                      [-0.0393,  1.6880, -1.1207, -0.1781],
                      [ 1.3211,  

In [28]:
%timeit Batch.cat(batches)

150 µs ± 289 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

323 µs ± 3.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
